In [30]:
import numpy as np

## n-gram Language Model

## Dataset

Before we get started, let's quickly process the dataset. The steps are:
1. Download the dataset to the `data/` directory first using: `curl -O https://raw.githubusercontent.com/karpathy/makemore/master/names.txt`
2. Open up the file and see what it looks like. It's a list of names, one per line.
3. Generate a random split of the data into training, validation, and test sets. We'll do 1000 names each for validation and test, and the rest for training.
4. Write each dataset as a separate file in the `data/` directory.

In [1]:
!curl -o data/names.txt https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  222k  100  222k    0     0   779k      0 --:--:-- --:--:-- --:--:--  795k


In [5]:
import random

# 2. Read in all the names (N=32,032 names in total)
names = open("./data/names.txt", "r").readlines()
print("First 10 names:")
for name in names[:10]:
    print(name.strip())

# 3. Get a permutation to split the names into test, val, and train sets
random.seed(42)  # fix seed for reproducibility
ix = list(range(len(names)))
random.shuffle(ix)

# (Validation, Test, Training) = (1000, 1000, N-2000)
test_names = [names[i] for i in ix[:1000]]
val_names = [names[i] for i in ix[1000:2000]]
train_names = [names[i] for i in ix[2000:]]


# 4. Write list of names to separate files
def write_names(names, filename):
    with open(filename, "w") as f:
        for name in names:
            f.write(name)


write_names(test_names, "./data/test.txt")
write_names(val_names, "./data/val.txt")
write_names(train_names, "./data/train.txt")


First 10 names:
emma
olivia
ava
isabella
sophia
charlotte
mia
amelia
harper
evelyn


## Tokenization

Now, we have a list of names. We need to convert them into a list of tokens (in this case, characters), so that we can train a character-level language model. More details about tokenization can be found in the [tokenization video on YouTube](https://www.youtube.com/watch?v=zduSFxRajkE).

In [6]:
train_text = open("data/train.txt", "r").read()

# Check that the text is as expected (a-z and a newline character)
assert all(c == "\n" or ("a" <= c <= "z") for c in train_text)


In [11]:
# Unique characters we see in the input
uchars = sorted(list(set(train_text)))
vocab_size = len(uchars)
char_to_token = {c: i for i, c in enumerate(uchars)}
token_to_char = {i: c for i, c in enumerate(uchars)}
# Designate \n as the delimiting <|endoftext|> token
EOT_TOKEN = char_to_token["\n"]

print(f"Number of unique characters: {vocab_size}")  # This should be 27
print(f"A -> : {char_to_token['a']}")
print(f"L -> : {char_to_token['l']}")
print(f"Z -> : {char_to_token['z']}")

Number of unique characters: 27
A -> : 1
L -> : 12
Z -> : 26


In [12]:
# Pre-tokenize all the splits one time up here
test_tokens = [char_to_token[c] for c in open("data/test.txt", "r").read()]
val_tokens = [char_to_token[c] for c in open("data/val.txt", "r").read()]
train_tokens = [char_to_token[c] for c in open("data/train.txt", "r").read()]

In [20]:
# Look at the first name
train_names[0]

'rayvon\n'

In [21]:
# Look at the first name in tokenized form
print([char_to_token[c] for c in train_names[0]])

# Look at the first few tokens of the training set
print(train_tokens[:10])

[18, 1, 25, 22, 15, 14, 0]
[18, 1, 25, 22, 15, 14, 0, 20, 1, 23]


As we can see above, the first name `rayvon\n` is tokenized into `['r', 'a', 'y', 'v', 'o', 'n', '\n']`, and then converted into numerical indices. Here, `0` represents the newline token `\n`.

## Build the n-gram model

The central idea behind an n-gram model is to approximate the probability of a token `w_n` given the history of the previous `n` tokens (denoted by `w_{n-N+1:n-1}`, more commonly known as the "context"). Here, the lowercase `n` refers to the position of the token in the sequence, while the uppercase `N` refers to the length of the context. For bigram models, `N=2`, for trigram models, `N=3`, and so on.

In mathematical terms, this is written as: $$ P(w_n | w_{n-N+1:n-1}) $$.

The above probability is estimated using maximum likelihood estimation (MLE) as: $$ P(w_n | w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1})} $$, where `C()` denotes the count of the n-gram in the dataset.

A more mathematical description can be found in the [n-gram Language Model](https://web.stanford.edu/~jurafsky/slp3/3.pdf) chapter of the book "Speech and Language Processing" by Dan Jurafsky & James H. Martin.

Let's build one such model using the training data using `N=3`

### Step-by-step walkthrough of training

In [50]:
seq_len = 3
counts = np.zeros((vocab_size,) * seq_len, dtype=np.int32)

print(counts.shape)


(27, 27, 27)


In [51]:
first_name = [char_to_token[c] for c in train_names[0]]
first_name


[18, 1, 25, 22, 15, 14, 0]

Since we have taken `N=3`, we will be building a trigram model. The model will be a dictionary where the keys are the context (a tuple of 2 tokens), and the values are also dictionaries. The inner dictionaries will have keys as the next token and values as the count of the n-gram in the dataset.

In [52]:
context = first_name[:seq_len]

print(f"Context: {context}")

counts[tuple(context)] += 1

print(f'Counts for context {[token_to_char[t] for t in context[:2]]}: {counts[tuple(context[:2])]}')


Context: [18, 1, 25]
Counts for context ['r', 'a']: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]


Above, we see that the counts for a given context ('r a') are updated to reflect the occurrence of the token 'y' (2nd last token in the vocabulary). In other words, we are saying: 
$$ C(y | r a) = 1 $$

This is the basic idea. To build a model, we just let this process run through the entire training dataset that is chunked into sizes of our sequence length `N`. Before we move on, let's see one more example.

In [55]:
context = first_name[1 : seq_len + 1]

print(f"Context: {context}")

counts[tuple(context)] += 1

print(
    f"Counts for context {[token_to_char[t] for t in context[:2]]}: {counts[tuple(context[:2])]}"
)

Context: [1, 25, 22]
Counts for context ['a', 'y']: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


Now, the count for the token 'v', given the context ('a y'), is increased by 1. In other words, we are saying: 
$$ C(v | a y) = 1 $$ 
I hope this gives you a good intuition about how the counts are updated. Let's now build the model using the training data.

### Training on the dataset

In [78]:
# Utility function to iterate tokens with a fixed-sized window
def dataloader(tokens, window_size):
    for i in range(len(tokens) - window_size + 1):
        yield tokens[i : i + window_size]

In [79]:
# Quick example:
example_dataloader = dataloader(train_tokens, 3)
print(1, next(example_dataloader))
print(2, next(example_dataloader))
print(3, next(example_dataloader))

1 [18, 1, 25]
2 [1, 25, 22]
3 [25, 22, 15]


In [80]:
counts = np.zeros((vocab_size,) * seq_len, dtype=np.int32)

# This will iterate over all trigrams in the training set and update the counts
for tape in dataloader(train_tokens, seq_len):
    counts[tuple(tape)] += 1

### Generating the next token using the model

Now that we have the trained model (an array of counts), we can generate the next token given a context. This is done by calculating the probability of the occurernce of each token given the context, and then sampling from this distribution to get the next token.

Let's look at an example, with context = `'a y'`.

In [86]:
inference_context_char = "ay"
inference_context = [char_to_token[c] for c in inference_context_char]
print("Inference context:", inference_context)

inference_counts = counts[tuple(inference_context)].astype(np.float32)

# Add smoothing ("fake counts") to all counts
inference_counts += 1

print(f"Counts for context ({inference_context_char}): {inference_counts}\n")

inference_counts_sum = inference_counts.sum()
probs = inference_counts / inference_counts_sum

print(f"Probs for context ({inference_context_char}): {probs}\n")
print(f"Most likely next character: {token_to_char[probs.argmax()]}")


Inference context: [1, 25]
Counts for context (ay): [158. 361.  14.  50. 172.  74.   4.  11.  10.  36.   9.  20. 451.  55.
  93.  47.   1.   5.  42. 129.  61.  20.  78.   2.   1.  16.  36.]

Probs for context (ay): [0.08077709 0.18456033 0.00715746 0.02556237 0.08793456 0.03783231
 0.00204499 0.00562372 0.00511247 0.01840491 0.00460123 0.01022495
 0.2305726  0.02811861 0.04754601 0.02402863 0.00051125 0.00255624
 0.02147239 0.06595092 0.03118609 0.01022495 0.0398773  0.00102249
 0.00051125 0.00817996 0.01840491]

Most likely next character: l


Let us look at another example, with context = `'k a'`.

In [87]:
inference_context_char = "ka"
inference_context = [char_to_token[c] for c in inference_context_char]
print("Inference context:", inference_context)

inference_counts = counts[tuple(inference_context)].astype(np.float32)

# Add smoothing ("fake counts") to all counts
inference_counts += 1

print(f"Counts for context ({inference_context_char}): {inference_counts}\n")

inference_counts_sum = inference_counts.sum()
probs = inference_counts / inference_counts_sum

print(f"Probs for context ({inference_context_char}): {probs}\n")
print(f"Most likely next character: {token_to_char[probs.argmax()]}")


Inference context: [11, 1]
Counts for context (ka): [199.  10.   4.  21.  40.  61.   1.   4.  52. 210.   6.   1. 164. 144.
  70.   5.   6.   1. 205. 106.  91.   7.  20.  10.   2. 195.  19.]

Probs for context (ka): [0.12031439 0.00604595 0.00241838 0.01269649 0.0241838  0.03688029
 0.00060459 0.00241838 0.03143894 0.12696493 0.00362757 0.00060459
 0.09915357 0.08706167 0.04232164 0.00302297 0.00362757 0.00060459
 0.12394196 0.06408706 0.05501814 0.00423216 0.0120919  0.00604595
 0.00120919 0.11789601 0.0114873 ]

Most likely next character: i


#### Sidenote: Smoothing

Smoothing involves adding a small value to all the counts in the model to ensure that no n-gram has a count of 0, which would lead to a division by zero error. The most common smoothing technique is called Laplace smoothing, where we add 1 to all the counts. See Section 3.6 of the aforementioned book for more details.

*Back to understanding n-grams...*

So, given a context of `'ay'`, the model predicts `'l'` as the most likely next token, while for the context `'ka'`, the model predicts `'i'` as the most likely next token.

How do we evaluate if this is correct?  